At first we import the required libraries namely numpy json and nltk. Then we import and read the json file that contains our intent.

In [ ]:
import json
import string
import random
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')

In [ ]:
data_root= 'C:/Username/File_location' #replace this with your directory
data_file = open(data_root + "/intents.json").read()
data = json.loads(data_file)


Below, we use nltk in order to organize the intent file into arrays. Then we lemmatize them, which is the proccess that turns all words into their basic root word form, which saves space and makes it easier for our model to learn.

In [ ]:
words = []
classes = []
dataX = []
dataY = []
for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens = nltk.word_tokenize(pattern)
    words.extend(tokens)
    dataX.append(pattern)
    dataY.append(intent["tag"])
  if intent["tag"] not in classes:
    classes.append(intent['tag'])
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))
classes = sorted(set(classes))


In [ ]:
training = []
out_empty = [0]* len(classes)
for idx, doc in enumerate(dataX):
  bow = []
  text = lemmatizer.lemmatize(doc.lower())
  for word in words:
    bow.append(1) if word in text else bow.append(0)
    output_row = list(out_empty)
    output_row[classes.index(dataY[idx])] = 1
    training.append([bow,output_row])
random.shuffle(training)
training = np.array(training, dtype=object)
trainX= np.array(list(training[:,0]))
trainY=  np.array(list(training[:,1]))

Here we create and train a simple neural network model.

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(trainX[0]),),activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(trainY[0]), activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01,decay=1e-6)
model.compile(loss = 'categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=trainX,y=trainY,epochs=150,verbose=1)

In [ ]:
def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text,vocab):
  tokens = clean_text(text)
  bow = [0]*len(vocab)
  for w in tokens:
    for idx,word in enumerate(vocab):
      if word == w:
        bow[idx]= 1
  return np.array(bow)

def pred_class(text,vocab,labels):
  bow = bag_of_words(text,vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.5
  y_pred = [[indx,res]for indx,res in enumerate(result) if res> thresh]
  y_pred.sort(key = lambda x: x[1],reverse = True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list,intents_json):
  if len(intents_list) == 0:
    result = "Sorry no bueno"
  else: 
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
  return result

Below is a simple code that allows us to interact with the finished chatbot model.

In [ ]:
print("press 0")
while True:
  message = input("")
  if message == "0":
    break
  intents = pred_class(message,words,classes)
  result =  get_response(intents,data)
  print(result)